In [ ]:
%matplotlib inline
import matplotlib
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.stats import chisquare
from astropy.cosmology import Planck15
from astropy import constants, units

def d_planck15(z):
    R0 = (constants.c)/(Planck15.H0)
    d_L = (Planck15.luminosity_distance(z))/R0.to(units.Mpc)
    plt.plot(z, d_L)
    R0=R0.to(units.lyr)/1e9
    return R0,d_L
z = np.arange(0.0,1.5,0.01)
R0_GLY,d_L=d_planck15(z)
R0_GLY_CMB=R0_GLY
print(R0_GLY)
HubbleGRData=pd.DataFrame()
HubbleGRData['z']=z
HubbleGRData['d']=d_L

# http://astronomy.swin.edu.au/cosmos/D/Distance+Modulus
SCPData = pd.ExcelFile('Data SupernovaLBLgov/SCPUnionData.xlsx').parse('Sheet2')
angle=math.pi/4
# First let's create a set of lambda-shifts
z = SCPData['redshift']

In [ ]:
H0=72*units.km/units.s/units.mpc
H0_saved=H0
R0Diff=R0_GLY-(constants.c/H0).to(units.lyr)
R0_GLY=(constants.c/H0).to(units.lyr)
R0_GLY

In [ ]:
H0=1
c=1
R0=1
def alphaZ(x):
    alpha = math.pi/4 - math.asin(1/math.sqrt(2)/(1+x))
    return alpha

def d_HU_epoch(R0,z):
    alpha = alphaZ(z)
    d_HU=R0*(1- math.cos(alpha)+math.sin(alpha))
    return d_HU

The distance d_HU is the arc on the outter circumference.  Later that distance will be converted on d_HU_corrected to reflect the point-to-point distance between our observation point and the position in the 4D Spatial manifold where the Supernova exploded.

Remember that

$$ \frac {\lambda_0}{\lambda_{Obs}}  = \frac{1}{1+z} = \sqrt2 sin(\frac{\pi}{4}-\alpha))$$

So
 $$  \alpha=  \frac{\pi}{4}- asin(\frac{1}{\sqrt2(1+z)})   )     $$


so
                    
                    alpha = [math.pi/4 - math.asin(/math.sqrt(2)/(1+x)) for x in z]

The scaling factor  $$G_f = \frac {R_0}{R(t)} = \frac {1}{(cos(\alpha)-sin(\alpha))}$$

The value of d', the actual distance is derived from the triangle law

$$\frac{sin(\alpha)}{d'} = \frac{sin(\frac{3\pi}{4})}{R_0} =  \frac{sin(\frac{\pi}{4} - \alpha)}{R(t)}$$


$$ {d'}=\sqrt2 R_0 sin(\alpha)= \sqrt2 R_0 sin(\frac{\pi}{4}-asin(\frac{1}{\sqrt2(1+z)})) $$

The distance between Universe epochs is what is traversed at the speed of light c on our expanding hypersphere and that is given by:

$$ d_{epoch}(t) = \frac {R_0-R(t)}{R_0} = 1 - \frac{1}{G_f}= 1-\frac {R(t)}{R_0} =1- cos(\alpha)+sin(\alpha)   $$

This value is the one used on the predictions of the Hypergeometrical Universe.  This is the distance travelled by a light entity traveling at the speed of light c, during the time the Universe took to reach our epoch.  This is equivalent to the distance d on our epoch.  This is the appropriate mapping since in the HU model, light travels at $\sqrt2 c$ within a 4D spatial manifold.

In [ ]:
# All the light coming from supernovae maps initially to the cosmological region [0,pi/4]
# no modifications done to initial data
d_bar_measured= SCPData["d"]/R0_GLY

In [ ]:
# Consider two points separated by d_HU on the current epoch (current hyperspherical hypersurface)

# First let's calculate the distance at the current Hypersphere. This distance will be projected into inner hyperspheres
# until a 45 degrees ray of light reaches the observer. That hypersphere will be defined by an R(t) and a d_HU_corrected.
# Simple geometry will be used to derive both.

alpha = [math.pi/4 - math.asin(1/math.sqrt(2)/(1+x)) for x in z]
Gf=[1/(math.cos(x)-math.sin(x)) for x in alpha]

# Calculate correction factors for measured distances
correctingFactors=[Gf[i]**1.5 for i in range(len(z))]
d_bar_measured_corrected=[d_bar_measured[i]/correctingFactors[i] for i in range(len(z))]


In [ ]:
# the 4D Line_of_sight distance d_prime is easily calculated as:
d_prime_4DLineOfSight = [R0*math.sqrt(2)*math.sin(x) for x in alpha]
# This distance (line of sight) is not relevant since the actual trajectory of light goes from layer to layer at 45 
# degrees, thus traversing the same distance radially as tangentially.  The distance traversed tangentially is equal 
# to d_HU_corrected calculated by d_HU_epoch or distance to epoch.
# Notice that all distances are derived from z -> alpha -> d_HU_epoch = R0(1-cos(alpha)+sin(alpha))
# where R0 is always 1
d_HU_corrected= [d_HU_epoch(1,x) for x in z]

In [ ]:
############################################
# Notice that all distances are derived from z -> alpha -> d = R0(1-cos(alpha)+sin(alpha))
# where R0 is always 1
upperlim=2-math.pi/4
bigz=[z*0.01 for z in range(2050)]
d_HU_big_corrected=pd.DataFrame()

d_HU_big_corrected['z']=[z for z in bigz]
d_HU_big_corrected['d']= [d_HU_epoch(1,x) for x in bigz]

d_HU_big_up_corrected=pd.DataFrame()
d_HU_big_up_corrected['z']=[z for z in bigz]
d_HU_big_up_corrected['d']=[2-x for x in d_HU_big_corrected.d]
ind= (d_HU_big_up_corrected.d<=upperlim) #& (d_HU_big_up_corrected.d>1) 
d_HU_big_up_corrected['d']=[x for x in d_HU_big_corrected.d]
d_HU_big_up_corrected=d_HU_big_up_corrected[ind]

In [ ]:
############################################
plt.semilogy(bigz,d_HU_big_corrected.d,'k-', label='1stQuad HU Hubble Law')
plt.semilogy(d_HU_big_up_corrected.z,d_HU_big_up_corrected.d,'r.', label='2ndQuad HU Hubble Law')
plt.semilogy(z,d_bar_measured_corrected,'k-', label='SCPData')
# plt.semilogy(z,d_prime_4DLineOfSight,'k+', label='4DLineOfSightDistance')


plt.xlabel('Relative Lambda Shift z')
plt.ylabel('Distance d as a Fraction of R0')
plt.title('HU Predictions - z vs Distance')
plt.grid(True)

legend = plt.legend(loc='lower right', shadow=True, fontsize='small')

# Put a nicer background color on the legend.
legend.get_frame().set_facecolor('#00FFCC')

plt.ylim(0.01,1)
plt.xlim(0,10.55)
plt.savefig("img/vcDistanceBothQuad.png")
plt.show()

In [ ]:
np.shape(bigz),np.shape(d_HU_big_corrected)

In [ ]:
############################################
plt.semilogy(bigz,d_HU_big_corrected.d,'k-', label='1stQuad HU Hubble Law')
plt.semilogy(z,d_bar_measured_corrected,'k-', label='SCPData')

plt.xlabel('Relative Lambda Shift z')
plt.ylabel('Distance d as a Fraction of R0')
plt.title('HU Predictions - z vs Distance')
plt.grid(True)

legend = plt.legend(loc='lower right', shadow=True, fontsize='small')

# Put a nicer background color on the legend.
legend.get_frame().set_facecolor('#00FFCC')
plt.ylim(0.01,1)
plt.xlim(0,1.5)
plt.savefig("img/vcDistancesmall.png")
plt.show()

In [ ]:
max(d_bar_measured_corrected)

In [ ]:
plt.plot(HubbleGRData.z,HubbleGRData.d,'k:', label='Friedmann-Lemaitre Fitting')
plt.plot(z,d_bar_measured,'k+', label='SCPData')
plt.plot(z,d_HU_corrected,'k.', label='HU Hubble Law')
# Correcting Stellar Candles for Gravitational R(t) dependence
# The Absolute Luminosity should scale down with G ~ (R0/R(t))^(3/2)

plt.plot(z,d_bar_measured_corrected,'k-', label='Normalized SCPData Epoch-Corrected')

plt.xlabel('Relative Lambda Shift z')
plt.ylabel('Distance d as a Fraction of R0')
plt.title('z vs Distance vs')
plt.grid(True)

legend = plt.legend(loc='upper left', shadow=True, fontsize='small')

# Put a nicer background color on the legend.
legend.get_frame().set_facecolor('#00FFCC')
# plt.ylim(0,2.4)
plt.xlim(0,1.4)
plt.savefig("img/vcDistance.png")
plt.show()

In [ ]:
relativeD_Error=[d_bar_measured_corrected[i]/d_HU_corrected[i]-1 for i in range(len(d_HU_corrected))]

plt.plot(z,relativeD_Error,'k-', label='Relative Distance Epoch-Corrected Error')
plt.xlabel('RedShift z')
plt.ylabel('Relative Error as a Fraction of distance')
plt.title('z vs Distance error')
plt.grid(True)

legend = plt.legend(loc='upper left', shadow=True, fontsize='small')

# Put a nicer background color on the legend.
legend.get_frame().set_facecolor('#00FFCC')
plt.ylim(-0.6,0.6)
plt.xlim(0,1.5)
plt.savefig("img/variance.png")
plt.show()

In [ ]:
plt.semilogy(z,d_bar_measured_corrected,'k-', label='Epoch-Corrected Distance ')
plt.semilogy(z,d_HU_corrected,'k-', label='HU Distance Prediction')

plt.xlabel('RedShift z')
plt.ylabel('Fractional Distance (fraction of R0)')
plt.title('z vs Distance for Supernova Survey Data')
plt.grid(True)

legend = plt.legend(loc='lower right', shadow=True, fontsize='small')

# Put a nicer background color on the legend.
legend.get_frame().set_facecolor('#00FFCC')
plt.ylim(0,1)
plt.xlim(0,1.4)
plt.savefig("img/comparison.png")
plt.show()

In [ ]:
chisquare(relativeD_Error)

The best fitting R0 for the Supernova dataset is for H0=72, that corresponds to R0=13.58.  The best for CMB is R0=14.43, so there is a difference of 0.85 Billion Light Years difference between these two measurements!!!



<h1>The segment below is related to the Baryonic Acoustic Oscillations. I wanted to see how my theory would deal with that. This is work in progress</h1>

<br>
<br>

In [ ]:
chisquare(d_HU_corrected, d_bar_measured_corrected)

In [ ]:
dbh=0.5*1.6737236e-27/(0.191753e-15)**3*units.kg/(units.m)**3
# dbh=6e18*units.kg/(units.m)**3
dbh

In [ ]:
dneutron=0.5*1.6737236e-27/(0.6e-15)**3*units.kg/(units.m)**3
# dneutron=4e17*units.kg/(units.m)**3
dneutron

In [ ]:
criticalDensity=9.47E-27*units.kg/(units.m)**3/25
criticalDensity

In [ ]:
nlayers=2e12
initialthickness = 0.19e-15*units.m
thickness = 2e12*initialthickness
Dilution=dbh/criticalDensity*nlayers
Dilution,thickness.to(units.mm)

In [ ]:
initialD0=4e13*dbh
initialD0

In [ ]:
Expansion=Dilution**(1/3)
Expansion

In [ ]:
TimeOfUniverse=13.58*1E9*365*24*3600*units.s
TimeOfTheUniverseCBM=14.43*1E9*365*24*3600*units.s
diffCBM=TimeOfTheUniverseCBM-TimeOfUniverse
TimeOfUniverse,diffCBM.to(units.yr)

In [ ]:
InitialTime=TimeOfUniverse/Expansion
InitialTime.to(units.ms)
RelaxedTime=nlayers*InitialTime
cooling=diffCBM/InitialTime
print(InitialTime.to(units.ms),RelaxedTime.to(units.yr),diffCBM.to(units.yr),cooling.round(0))

In [ ]:
InitialRadius=constants.c*InitialTime
InitialRadius.to(units.km)

In [ ]:
Lambda=constants.c*units.s/111101
Lambda.to(units.km)
G=6.67408E-11*units.m**3/units.kg/units.s**2
G,dbh

In [ ]:
f=G*dbh/4/math.pi*Expansion
f

In [ ]:
fractionalLambda=Lambda/InitialRadius*180/math.pi/0.1
print('initialLambda=',fractionalLambda)


In [ ]:
zOneRadian=-1/math.sin(math.pi/4-1)-1
print(zOneRadian)

In [ ]:
print(H0_saved,R0_GLY,R0_GLY_CMB, R0Diff)